In [3]:
import pandas as pd

In [4]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def laliga_url(start_year):
    end_short = str(start_year + 1)[-2:]
    return f"https://en.wikipedia.org/wiki/{start_year}%E2%80%93{end_short}_La_Liga"

In [13]:
from io import StringIO

# Add headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def scrape_legue_table(start_year):
    url = laliga_url(start_year)
    
    # Fetch the page with headers
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    # Parse tables from the HTML content
    tables = pd.read_html(StringIO(response.text), attrs={"class": "wikitable"})
    league_table = tables[0]
    return league_table
    

In [ ]:
df_2014_2015 = scrape_legue_table(2014)
df_2014_2015

In [15]:
df_2014_2015 = scrape_legue_table(2014)
df_2015_2016 = scrape_legue_table(2015)
df_2016_2017 = scrape_legue_table(2016)
df_2017_2018 = scrape_legue_table(2017)
df_2018_2019 = scrape_legue_table(2018)
df_2019_2020 = scrape_legue_table(2019)
df_2020_2021 = scrape_legue_table(2020)
df_2021_2022 = scrape_legue_table(2021)
df_2022_2023 = scrape_legue_table(2022)
df_2023_2024 = scrape_legue_table(2023)
df_2024_2025 = scrape_legue_table(2024)
df_2025_2026 = scrape_legue_table(2025)

In [21]:
df_2021_2022

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification or relegation
0,1,Real Madrid (C),38,26,8,4,80,31,+49,86,Qualification for the Champions League group s...
1,2,Barcelona,38,21,10,7,68,38,+30,73,Qualification for the Champions League group s...
2,3,Atlético Madrid,38,21,8,9,65,43,+22,71,Qualification for the Champions League group s...
3,4,Sevilla,38,18,16,4,53,30,+23,70,Qualification for the Champions League group s...
4,5,Real Betis,38,19,8,11,62,40,+22,65,Qualification for the Europa League group stag...
5,6,Real Sociedad,38,17,11,10,40,37,+3,62,Qualification for the Europa League group stag...
6,7,Villarreal,38,16,11,11,63,37,+26,59,Qualification for the Europa Conference League...
7,8,Athletic Bilbao,38,14,13,11,43,36,+7,55,NaN
8,9,Valencia,38,11,15,12,48,53,−5,48,NaN
9,10,Osasuna,38,12,11,15,37,51,−14,47,NaN
